In [1]:
from unsloth import FastModel
import torch

dtype = None
load_in_4bit = False 

max_seq_length = 2048

model, tokenizer = FastModel.from_pretrained(
    model_name = "OuteAI/Llama-OuteTTS-1.0-1B",
    max_seq_length= max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
    )

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/adrian/anaconda3/envs/llm-fine-tuning/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce GTX 1660 SUPER. Num GPUs = 1. Max memory: 5.606 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


In [2]:
#LoRA does not work with float32 only works with bfloat16 !!!
model = FastModel.get_peft_model(
    model,
    r = 128, 
    target_modules = ["q_proj", "v_proj",],
    lora_alpha = 128,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False, 
    loftq_config = None,
)

Unsloth: Making `model.base_model.model.model` require gradients


In [3]:
from datasets import load_dataset,Audio,Dataset
dataset = load_dataset("MrDragonFox/Elise", split = "train")
dataset = dataset.cast_column("audio", Audio(sampling_rate=24000))


In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2): 100%|██████████| 1195/1195 [00:02<00:00, 407.73 examples/s]


In [6]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,195 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 13,631,488/1,262,028,800 (1.08% trained)


Step,Training Loss
1,4.508600
2,4.174500
3,4.667600
4,4.629500
5,3.995500
6,4.801800
7,4.423600
8,4.047900
9,3.992800
10,3.870000


In [8]:
import torch
import re
import numpy as np
from typing import Dict, Any
import torchaudio.transforms as T
from transformers import LogitsProcessor
import transformers.generation.utils as generation_utils
from transformers import AutoModelForCausalLM
import re

def get_audio(tokens):
        decoded_output = tokenizer.batch_decode(tokens, skip_special_tokens=False)[0]
        c1 = list(map(int,re.findall(r"<\|c1_(\d+)\|>", decoded_output)))
        c2 = list(map(int,re.findall(r"<\|c2_(\d+)\|>", decoded_output)))

        t = min(len(c1), len(c2))
        c1 = c1[:t]
        c2 = c2[:t]
        output = [c1,c2]
        if not output:
            print("No audio tokens found in the output")
            return None

        return data_processor.audio_processor.audio_codec.decode(
            torch.tensor([output], dtype=torch.int64).to(data_processor.audio_processor.audio_codec.device)
        )

class RepetitionPenaltyLogitsProcessorPatch(LogitsProcessor):
    def __init__(self, penalty: float):
        penalty_last_n = 64
        print("🔄 Using patched RepetitionPenaltyLogitsProcessor -> RepetitionPenaltyLogitsProcessorPatch | penalty_last_n: {penalty_last_n}")
        if penalty_last_n is not None:
            if not isinstance(penalty_last_n, int) or penalty_last_n < 0:
                raise ValueError(f"`penalty_last_n` has to be a non-negative integer, but is {penalty_last_n}")
        if not isinstance(penalty, float) or penalty <= 0:
            raise ValueError(f"`penalty` has to be a positive float, but is {penalty}")

        self.penalty_last_n = penalty_last_n
        self.penalty = penalty

    @torch.no_grad()
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor) -> torch.FloatTensor:
        """
        Args:
            input_ids (`torch.LongTensor`):
                Indices of input sequence tokens in the vocabulary (shape `(batch_size, sequence_length)`).
            scores (`torch.FloatTensor`):
                Prediction scores of a language modeling head (shape `(batch_size, vocab_size)`).

        Returns:
            `torch.FloatTensor`: The modified prediction scores.
        """
        # Check if penalties should be applied
        if self.penalty_last_n == 0 or self.penalty == 1.0:
            return scores

        batch_size, seq_len = input_ids.shape
        vocab_size = scores.shape[-1]

        # Process each batch item independently
        for b in range(batch_size):
            # 1. Determine the penalty window
            start_index = max(0, seq_len - self.penalty_last_n)
            window_indices = input_ids[b, start_index:] # Shape: (window_len,)

            if window_indices.numel() == 0: # Skip if window is empty
                continue

            # 2. Find unique tokens within the window
            tokens_in_window = set(window_indices.tolist())

            # 3. Apply repetition penalty to the scores for this batch item
            for token_id in tokens_in_window:
                if token_id >= vocab_size:
                    continue

                logit = scores[b, token_id]

                if logit <= 0:
                    logit *= self.penalty
                else:
                    logit /= self.penalty

                # Update the score
                scores[b, token_id] = logit

        return scores

generation_utils.RepetitionPenaltyLogitsProcessor = RepetitionPenaltyLogitsProcessorPatch
AutoModelForCausalLM.generate = generation_utils.GenerationMixin.generate

if __name__ == "__main__":
    formated_text = "<|text_start|>"+input_text+"<|text_end|>"
    prompt = "\n".join([
        "<|im_start|>",
        formated_text,
        "<|audio_start|><|global_features_start|>",
    ])

    model_inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    print("Generating token sequence...")
    generated_ids = model.generate(
        **model_inputs,
        temperature=0.4,
        top_k=40,
        top_p=0.9,
        repetition_penalty=1.1,
        min_p=0.05,
        max_new_tokens=2048, # Limit generation length
    )
    print("Token sequence generated.")


    generated_ids_trimmed = generated_ids[:, model_inputs.input_ids.shape[1]:]
    audio = get_audio(generated_ids)
    audio = audio.cpu()
    from IPython.display import Audio, display
    display(Audio(audio.squeeze(0), rate=24000))


OSError: /home/adrian/anaconda3/envs/llm-fine-tuning/lib/python3.11/site-packages/torchaudio/lib/libtorchaudio.so: undefined symbol: _ZNK5torch8autograd4Node4nameB5cxx11Ev